# Introduction
Two most popular areas in Machine Learning are Supervised Learning & Unsupervised Learning. From solutions perspective the two areas are quite similar, as they basically give a single output given some data. But not all problems are so closely circuited and might be more complex. Lot of solutions are also deployed where complex problems are broken down to ML Pipelines, where different teams may be deployed to each component. The pipeline components are therefore generally independent of each other and talk to each other only as per solution defined APIs. Here we introduce Reinforcement Learning, which in a witty sense can be explained as Machine Learning Pipeline built using Machine Learning.   

Reinforcement Learning is an emerging Machine Learning Area, where by we identify incremental 
decisions that may finally lead us to a success critria. This kind of application is most easily understood
in the gaming arena. Some interesting games being studied in this area are:
1. Go [3]
2. Chess [4]
3. Tic Tac Toe [2]
4. Mario [5]
and many more. 

The gaming industry is not the only one with reinforcement learning usecases, but all feilds with strategic intelligence requirements are expected to be greatly influenced by it. Be it Stock Trading, Autonomous Driving or maybe Robotic Surgeries (some day). If machines can become good at strategizing someday, it would really makes the jobs of many white collar executives obsolete.  

Another way far-fetched yet interesting application of reinforcement learning can be to to create alter-images for people like one described at [6].

# Q Learning Algorithm
Now this is a pretty introductory session and my goal is to introduce a very simple and powerful reinforcement learning algorithm - Q Learning. To begin with all courtesies of this blog to Edureka for this excellent video on Youtube [1]. Let us try to learn a model that can come out of a simple maze. Here is the simple maze and a graph based representation of the same maze at https://github.com/HimanshuGautam17/Personal-Data-Repo/blob/master/Toy%20Maze.JPG (Not able to add a image in a Jupyter notebook hosted at github as of now ;)). 

This algorithm aims at coming out with best "list of decisions" to reach the end of maze given a random starting position in the graph. The algorithm depends on 2 matrices R & Q. 
R (Reward) matrix bascailly models the rules of the game. We create a square matrix of size equal to possible number of states and then define which states are favourable. So the states which directly lead to State "5" (out of maze) are given 100 points. All invalid state transitions are labelled -1 and other transitions are labelled as 0.   

In [3]:
import numpy as np
# 6 dimensions for State 0 to 5
R = np.matrix([[-1, -1, -1, -1, 0, -1],
               [-1, -1, -1, 0, -1, 100],
               [-1, -1, -1, 0, -1, -1],
               [-1, 0, 0, -1, 0, -1],
               [-1, 0, 0, -1, -1, 100],
               [-1, 0, -1, -1, 0, 100]])

Q matrix is a square matrix with dimension same as R. Here the incremental value of taking an action (i to j) is stored. This matrix is initially all zero and is filled based on the experience of agent by taking different paths.

In [4]:
#Q Matrix
Q = np.zeros([6,6])

# Exploitation vs Exploration (Hyper parameter)
Gamma is a learning parameter that defines the explorative vs exploitative nature of the agent. A high gamma means agent gives more weightage to already learned Q matrix values (known paths / incremental reward), while low value means more weightage to exploring new paths. Here we set it as 0.8. The parameter does not seem to be much important for this toy example, but will be important for more complex problems. 

The method to update Q matrix is:<br><br>
**Q(current_state, action) = R(current_state, action) + gamma * Max(Q(action, all valid next actions))**<br><br>
The term R(current_state, action) represents explorative nature, where Reward is decided as per the rules of game. This value will be 0 if the action does not lead to state "5" and 100 otherwise. 
The Term Max(Q(action, all valid next actions)) represent the exploitative nature, where the reward for current action is determined by the max possible known reward from the state we would reach after taking the current action. So a path leading to known high rewards (based on experience) has a higher reward for itself.   

In [5]:
#Gamma - Learning parameter. 
gamma = 0.8

# Intial State - choosen randomly later on while training
initial_state = 1

Now we define some utility functions that returns list of valid actions from a initial_state. Choosing a random action from the valid actions and updating the Q matrix as per the gamma parameter.  

In [6]:
# Get available actions given a state
def available_actions(state):
    curr_state_row = R[state, ]
    av_act = np.where(curr_state_row >= 0)[1]
    return av_act

available_act = available_actions(initial_state)


#randomly choose action among available
def sample_next_action(avl_acts):
    next_action = int(np.random.choice(avl_acts, 1))
    return next_action

next_act = sample_next_action(available_act)

#This method updates Q matrix based on path choosen and Q Learning algo
def update(current_state, action, gamma):
    #print(action)
    max_index = np.where(Q[action,] == np.max(Q[action,]))[0]
    
    if max_index.shape[0] > 1:
        max_index = int(np.random.choice(max_index, 1))
    else:
        max_index = int(max_index)
        
    max_value = Q[action, max_index]
    
    #Q Learning Formula
    Q[current_state, action] = R[current_state, action] + gamma * max_value

#Update Q matrix
update(initial_state, next_act, gamma)


Now we have defined the utility methods, it is time to play this game a lot of times from random initial states. The Q Matrix is updated with all these iterations of game play and would represent the incremental rewards of taking any particular action at a given state. This Q matrix represents the model which shall be used when the
agent plays the game during testing phase. 

But before training , let us play the game with default (all Zero) Q Matrix:

In [7]:
#%% Testing Phase with Default Q Matrix
test_state = 2
steps = [test_state]
while(test_state != 5):
    next_step_index = np.where(Q[test_state, ] == np.max(Q[test_state,]))[0]
    if next_step_index.shape[0] > 1:
        next_step_index = int(np.random.choice(next_step_index, 1))
    else:
        next_step_index = int(next_step_index)
    
    steps.append(next_step_index)
    test_state = next_step_index

print(steps)

[2, 0, 4, 5]


All the steps shown above might not be even valid, as once trained we will only be referring to Q Matrix to make our decisions. Since Q matrix is all zero as of now, all state transitions are equally likely. Simply speaking the model does not know the rules of the game as of now. 

In [8]:
#%% Training
for i in range(10000):
    initial_state = int(np.random.randint(0, Q.shape[0]))
    available_act = available_actions(initial_state)
    action = sample_next_action(available_act)
    update(initial_state, action, gamma)


#Normalize Q Matrix
Q = Q / np.max(Q) * 100

Let us show the model in action now. 

In [9]:
#%% Testing Phase
test_state = 2
steps = [test_state]
while(test_state != 5):
    next_step_index = np.where(Q[test_state, ] == np.max(Q[test_state,]))[0]
    if next_step_index.shape[0] > 1:
        next_step_index = int(np.random.choice(next_step_index, 1))
    else:
        next_step_index = int(next_step_index)
    
    steps.append(next_step_index)
    test_state = next_step_index

print(steps)

[2, 3, 4, 5]


The path suggested by the model, after training is the best possible path. This completes the introduction of the Q Learn algorithm. 

After Thoughts:
This was a simple toy example, where only a small number of states were possible. Therefore we were able to describe the complete list of prescribed actions fairly easily. In most scenarios the number of states are huge and creating fully descriptive Q matrix is mostly impracticle for them. For example a tic tac toe game, which is again a very simple example can have 19683(3^9) possible states. As Per [2], Chess has 10^123 possible games, while Go has 10^360 possible games. In these scenerios, some plausible workarounds are:
1. we create Q matrix for a small number of steps ahead instead of end of game. 
2. Simulating large number of games using Monte Carlo Simulations and check what percetanges of games were won (or lost) from a given board position (State). 
3. Discounted Rewards may be applied backwards to a action that resulted in Reward. 
4. Deep Neural Networks are very common in predicting in the next move, if lot of game playing data is available. 
Some interesting resources to learn about advanced RL techniques are [7] & [8]. 

# References:
1. Edureka Video on Reinforcement Learning: https://www.youtube.com/watch?v=LzaWrmKL1Z4
2. Tic Tac Toe Project using Reinforcement Learning: https://danielslater.net/2016/10/01/alphatoe/
3. Deepmind's Alpha Go: https://deepmind.com/research/case-studies/alphago-the-story-so-far
4. Chess Project using reinforcement leanring: https://github.com/Zeta36/chess-alpha-zero
5. Mario Game Reinforcement Learning: https://www.cse.iitb.ac.in/~nilesh/media/Mario_Neat.pdf
6. Superman Fantasy AI: https://superman.fandom.com/wiki/Jor-El_A.I
7. Spinning Up: https://spinningup.openai.com/en/latest/index.html
8. Blog: https://lilianweng.github.io/lil-log/2018/02/19/a-long-peek-into-reinforcement-learning.html#key-concepts